In [2]:
import json 

data = json.load(open('./output.json'))
json.dump(data, open('./output.json', 'w'), indent=4, ensure_ascii=False)

In [1]:
### 生成EE的数据集
import numpy as np
import json
data = json.load(open('./data/ee/data_label.json'))
to_save=[]
for dialog in data:
    for turn in dialog['content']:
        keys = list(turn.keys())[:2]
        t_ents = turn['info']['ents']
        
        item1={
            'sentence':' '.join(turn[keys[0]]).split(),
            'ner':[]
        }
        item2={
            'sentence':' '.join(turn[keys[1]]).split(),
            'ner':[]
        }

        for ent in t_ents:
            ner = {
                'index':np.arange(ent['pos'][0][1], ent['pos'][0][2]).tolist(),
                'type':ent['type']
            }

            if ent['pos'][0][0] == 1:
                item1['ner'].append(ner)
            elif ent['pos'][0][0] == 2:
                item2['ner'].append(ner)
        
        if item1['ner']:
            to_save.append(item1)
        if item2['ner']:
            to_save.append(item2)

np.random.seed(0)
np.random.shuffle(to_save)
np.random.shuffle(to_save)
n = len(to_save)
json.dump(to_save[:int(.8*n)], open('./data/seretod/train.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
json.dump(to_save[int(.8*n):int(.9*n)], open('./data/seretod/dev.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
json.dump(to_save[int(.9*n):], open('./data/seretod/test.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
# json.dump(to_save, open('./data/all_data.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)


In [8]:
### 用基线的测试方法测试此模型的输出结果
task = 'sf'
import json 
import re
labels = json.load(open(f'./data/{task}/test.json', encoding='utf8'))
bio_l = []
texts = []
for sent in labels:
    texts.append(''.join(sent['sentence']))
    res = ['O']*len(sent['sentence'])
    for ent in sent['ner']:
        for i in ent['index']:
            res[i] = f'I-{ent["type"]}'
        res[ent['index'][0]] = f'B-{ent["type"]}'

    bio_l.append(res)

output = json.load(open(f'./output/{task}/output.json', encoding='utf8'))
bio_o = []
for sent in output:
    res = ['O']*len(sent['sentence'])
    text = ''.join(sent['sentence'])
    for ent in sent['entity']:
        label = ''.join(ent['text'])
        span = re.search(label, text).span()
        for i in range(*span):
            res[i] = f"I-{ent['type'].upper()}"
        res[span[0]] = f"B-{ent['type'].upper()}"
    bio_o.append(res)


from seqeval.metrics import f1_score as span_f1_score
from seqeval.scheme import IOB2
micro_f1 = span_f1_score(bio_l, bio_o, mode='strict', scheme=IOB2) * 100.0
micro_f1


63.71959678165171

In [11]:
cases = []
for text, label, output in zip(texts, bio_l, bio_o):
    if label!=output:
        cases.append({
            'text': text,
            'label': ' '.join(label),
            'output': ' '.join(output)
        })
json.dump(cases, open('./case.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)


In [6]:
### 处理生成SF的数据集
import numpy as np
import json
data = json.load(open('./data/ee/data_label.json'))
to_save=[]
for dialog in data:
    for turn in dialog['content']:
        keys = list(turn.keys())[:2]
        t_triples = turn['info']['triples']
        
        item1={
            'sentence':' '.join(turn[keys[0]]).split(),
            'ner':[]
        }
        item2={
            'sentence':' '.join(turn[keys[1]]).split(),
            'ner':[]
        }

        for triple in t_triples:
            ner = {
                'index':np.arange(triple['pos'][1], triple['pos'][2]).tolist(),
                'type':triple['prop']
            }

            if triple['pos'][0] == 1:
                item1['ner'].append(ner)
            elif triple['pos'][0] == 2:
                item2['ner'].append(ner)
        
        if item1['ner']:
            to_save.append(item1)
        if item2['ner']:
            to_save.append(item2)

np.random.seed(0)
np.random.shuffle(to_save)
np.random.shuffle(to_save)
n = len(to_save)
json.dump(to_save[:int(.8*n)], open('./data/sf/train.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
json.dump(to_save[int(.8*n):int(.9*n)], open('./data/sf/dev.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
json.dump(to_save[int(.9*n):], open('./data/sf/test.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
# json.dump(to_save, open('./data/all_data.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)

